# Introduction to Software Testing

Content and code adapted from fuzzingbook.org and shared under the same license conditions. [License](https://github.com/uds-se/fuzzingbook/blob/master/LICENSE.md)

Install the fuzzingbook package for python through pip.





In [ ]:
!pip install fuzzingbook

## Simple Testing

Let us start with a simple example.  Your co-worker has been asked to implement a square root function $\sqrt{x}$.  (Let's assume for a moment that the environment does  not already have one.)  After studying the [Newton–Raphson method](https://en.wikipedia.org/wiki/Newton%27s_method), she comes up with the following Python code, claiming that, in fact, this `my_sqrt()` function computes square roots.

In [ ]:
def my_sqrt(x):
    """Computes the square root of x, using the Newton-Raphson method"""
    approx = None
    guess = x / 2
    while approx != guess:
        approx = guess
        guess = (approx + x / approx) / 2
    return approx

Your job is now to find out whether this function actually does what it claims to do.

### Running a Function

To find out whether `my_sqrt()` works correctly, we can *test* it with a few values.  For `x = 4`, for instance, it produces the correct value:

In [ ]:
my_sqrt(4)

The upper part above `my_sqrt(4)` (a so-called _cell_) is an input to the Python interpreter, which by default _evaluates_ it.  The lower part (`2.0`) is its output.  We can see that `my_sqrt(4)` produces the correct value.

The same holds for `x = 2.0`, apparently, too:

In [ ]:
my_sqrt(9)

### Debugging a Function

To see how `my_sqrt()` operates, a simple strategy is to insert `print()` statements in critical places.  You can, for instance, log the value of `approx`, to see how each loop iteration gets closer to the actual value:

In [ ]:
def my_sqrt_with_log(x):
    """Computes the square root of x, using the Newton–Raphson method"""
    approx = None
    guess = x / 2
    while approx != guess:
        print("approx =", approx)  # <-- New
        approx = guess
        guess = (approx + x / approx) / 2
    return approx

In [ ]:
my_sqrt_with_log(9)

### Checking a Function

Let's get back to testing.  We can read and run the code, but are the above values of `my_sqrt(2)` actually correct?  We can easily verify by exploiting that $\sqrt{x}$ squared again has to be $x$, or in other words $\sqrt{x} \times \sqrt{x} = x$.  Let's take a look:

In [ ]:
my_sqrt(2) * my_sqrt(2)

Okay, we do have some rounding error, but otherwise, this seems just fine.

What we have done now is that we have _tested_ the above program: We have _executed_ it on a given input and _checked_ its result whether it is correct or not.  Such a test is the bare minimum of quality assurance before a program goes into production.

## Automating Test Execution

So far, we have tested the above program _manually_, that is, running it by hand and checking its results by hand.  This is a very flexible way of testing, but in the long run, it is rather inefficient:

1. Manually, you can only check a very limited number of executions and their results
2. After any change to the program, you have to repeat the testing process

This is why it is very useful to _automate_ tests.  One simple way of doing so is to let the computer first do the computation, and then have it check the results.

For instance, this piece of code automatically tests whether $\sqrt{4} = 2$ holds:

In [ ]:
result = my_sqrt(4)
expected_result = 2.0
if result == expected_result:
    print("Test passed")
else:
    print("Test failed")

The nice thing about this test is that we can run it again and again, thus ensuring that at least the square root of 4 is computed correctly.  But there are still a number of issues, though:

1. We need _five lines of code_ for a single test
2. We do not care for rounding errors
3. We only check a single input (and a single result)

Let us address these issues one by one.  First, let's make the test a bit more compact.  Almost all programming languages do have a means to automatically check whether a condition holds, and stop execution if it does not.  This is called an _assertion_, and it is immensely useful for testing.

In Python, the `assert` statement takes a condition, and if the condition is true, nothing happens.  (If everything works as it should, you should not be bothered.)  If the condition evaluates to false, though, `assert` raises an exception, indicating that a test just failed.

In our example, we can use `assert` to easily check whether `my_sqrt()` yields the expected result as above:

In [ ]:
assert my_sqrt(4) == 2

As you execute this line of code, nothing happens: We just have shown (or asserted) that our implementation indeed produces $\sqrt{4} = 2$.

Remember, though, that floating-point computations may induce rounding errors.  So we cannot simply compare two floating-point values with equality; rather, we would ensure that the absolute difference between them stays below a certain threshold value, typically denoted as $\epsilon$ or ``epsilon``.  This is how we can do it:

In [ ]:
EPSILON = 1e-8

In [ ]:
assert abs(my_sqrt(4) - 2) < EPSILON

We can also introduce a special function for this purpose, and now do more tests for concrete values:

In [ ]:
def assertEquals(x, y, epsilon=1e-8):
    assert abs(x - y) < epsilon

In [ ]:
assertEquals(my_sqrt(4), 2)
assertEquals(my_sqrt(9), 3)
assertEquals(my_sqrt(100), 10)

Seems to work, right?  If we know the expected results of a computation, we can use such assertions again and again to ensure our program works correctly.

## Generating Tests

Remember that the property $\sqrt{x} \times \sqrt{x} = x$ universally holds?  We can also explicitly test this with a few values:

In [ ]:
assertEquals(my_sqrt(2) * my_sqrt(2), 2)
assertEquals(my_sqrt(3) * my_sqrt(3), 3)
assertEquals(my_sqrt(42.11) * my_sqrt(42.11), 42.11)

Still seems to work, right?  Most importantly, though, $\sqrt{x} \times \sqrt{x} = x$ is something we can very easily test for thousands of values:

In [ ]:
for n in range(1, 1000):
    assertEquals(my_sqrt(n) * my_sqrt(n), n)

## Run-Time Verification

Instead of writing and running tests for `my_sqrt()`, we can also go and _integrate the check right into the implementation._  This way, _each and every_ invocation of `my_sqrt()` will be automatically checked.

Such an _automatic run-time check_ is very easy to implement:

In [ ]:
def my_sqrt_checked(x):
    root = my_sqrt(x)
    assertEquals(root * root, x)
    return root

Now, whenever we compute a root with `my_sqrt_checked()`$\dots$

In [ ]:
my_sqrt_checked(2.0)

we already know that the result is correct, and will so for every new successful computation.

Automatic run-time checks, as above, assume two things, though:

* One has to be able to _formulate_ such run-time checks.  Having concrete values to check against should always be possible, but formulating desired properties in an abstract fashion can be very complex.  In practice, you need to decide which properties are most crucial, and design appropriate checks for them.  Plus, run-time checks may depend not only on local properties, but on several properties of the program state, which all have to be identified.

* One has to be able to _afford_ such run-time checks.  In the case of `my_sqrt()`, the check is not very expensive; but if we have to check, say, a large data structure even after a simple operation, the cost of the check may soon be prohibitive.  In practice, run-time checks will typically be disabled during production, trading reliability for efficiency.  On the other hand, a comprehensive suite of run-time checks is a great way to find errors and quickly debug them; you need to decide how many such capabilities you would still want during production.

## System Input vs Function Input

At this point, we may make `my_sqrt()` available to other programmers, who may then embed it in their code.  At some point, it will have to process input that comes from _third parties_, i.e. is not under control by the programmer.

Let us simulate this *system input* by assuming a _program_ `sqrt_program()` whose input is a string under third-party control:

In [ ]:
def sqrt_program(arg):
    x = int(arg)
    print('The root of', x, 'is', my_sqrt(x))

We assume that `sqrt_program` is a program which accepts system input from the command line, as in

```shell
$ sqrt_program 4
2
```

We can easily invoke `sqrt_program()` with some system input:

In [ ]:
sqrt_program("4")

What's the problem?  Well, the problem is that we do not check external inputs for validity.  Try invoking `sqrt_program(-1)`, for instance.  What happens?

Indeed, if you invoke `my_sqrt()` with a negative number, it enters an infinite loop.  For technical reasons, we cannot have infinite loops in this chapter (unless we'd want the code to run forever); so we use a special `with ExpectTimeOut(1)` construct to interrupt execution after one second.

In [ ]:
from fuzzingbook.ExpectError import ExpectTimeout

In [ ]:
with ExpectTimeout(1):
    sqrt_program("-1")

The above message is an _error message_, indicating that something went wrong.  It lists the *call stack* of functions and lines that were active at the time of the error.  The line at the very bottom is the line last executed; the lines above represent function invocations – in our case, up to `my_sqrt(x)`.

We don't want our code terminating with an exception.  Consequently, when accepting external input, we must ensure that it is properly validated.  We may write, for instance:

In [ ]:
def sqrt_program(arg):
    x = int(arg)
    if x < 0:
        print("Illegal Input")
    else:
        print('The root of', x, 'is', my_sqrt(x))

and then we can be sure that `my_sqrt()` is only invoked according to its specification.

In [ ]:
sqrt_program("-1")

But wait!  What happens if `sqrt_program()` is not invoked with a number?  Then we would try to convert a non-number string, which would also result in a runtime error:

In [ ]:
from fuzzingbook.ExpectError import ExpectError

In [ ]:
with ExpectError():
    sqrt_program("xyzzy")

Here's a version which also checks for bad inputs:

In [ ]:
def sqrt_program(arg):
    try:
        x = float(arg)
    except ValueError:
        print("Illegal Input")
    else:
        if x < 0:
            print("Illegal Number")
        else:
            print('The root of', x, 'is', my_sqrt(x))

In [ ]:
sqrt_program("4")

In [ ]:
sqrt_program("-1")

In [ ]:
sqrt_program("xyzzy")

We have now seen that at the system level, the program must be able to handle any kind of input gracefully without ever entering an uncontrolled state.  This, of course, is a burden for programmers, who must struggle to make their programs robust for all circumstances.  This burden, however, becomes a _benefit_ when generating software tests: If a program can handle any kind of input (possibly with well-defined error messages), we can also _send it any kind of input_.  When calling a function with generated values, though, we have to _know_ its precise preconditions.

## The Limits of Testing

Despite best efforts in testing, keep in mind that you are always checking functionality for a _finite_ set of inputs.  Thus, there may always be _untested_ inputs for which the function may still fail.

In the case of `my_sqrt()`, for instance, computing $\sqrt{0}$ results in a division by zero:

In [ ]:
with ExpectError():
    root = my_sqrt(0)

In our tests so far, we have not checked this condition, meaning that a program which builds on $\sqrt{0} = 0$ will surprisingly fail.  But even if we had set up our random generator to produce inputs in the range of 0–1000000 rather than 1–1000000, the chances of it producing a zero value by chance would still have been one in a million.  If the behavior of a function is radically different for few individual values, plain random testing has few chances to produce these.

We can, of course, fix the function accordingly, documenting the accepted values for `x` and handling the special case `x = 0`:

In [ ]:
def my_sqrt_fixed(x):
    assert 0 <= x
    if x == 0:
        return 0
    return my_sqrt(x)

With this, we can now correctly compute $\sqrt{0} = 0$:

In [ ]:
assert my_sqrt_fixed(0) == 0

Illegal values now result in an exception:


In [ ]:
with ExpectError():
    root = my_sqrt_fixed(-1)

Still, we have to remember that while extensive testing may give us a high confidence into the correctness of a program, it does not provide a guarantee that all future executions will be correct.  Even run-time verification, which checks every result, can only guarantee that _if_ it produces a result, the result will be correct; but there is no guarantee that future executions may not lead to a failing check.  As I am writing this, I _believe_ that `my_sqrt_fixed(x)` is a correct implementation of $\sqrt{x}$ for all finite numbers $x$, but I cannot be certain.

## Lessons Learned

* The aim of testing is to execute a program such that we find bugs.
* Test execution, test generation, and checking test results can be automated.
* Testing is _incomplete_; it provides no 100% guarantee that the code is free of errors.